In [1]:
import os
from pycocotools.coco import COCO
from transformers import AutoImageProcessor, AutoModelForObjectDetection, Trainer
from functools import partial
import argparse
import utils
import dataset
import train_eval
from glob import glob

In [10]:
current_path = ''
exp_conf_path = '/data/ephemeral/home/Dongjin/level2-objectdetection-cv-07/transformers/Dongjin/1019_resume_train/config/exp1.json'
default_conf_path = os.path.join(current_path, 'config/default.json')
conf = utils.load_conf(default_conf_path, exp_conf_path)
checkpoint_path = '/data/ephemeral/home/Dongjin/level2-objectdetection-cv-07/transformers/Dongjin/1019_resume_train/result/deta-swin-large_3_img_size_720/checkpoint-15704'

conf['output_dir'] = os.path.join(current_path, 'result/' + conf['output_dir_format'].format(**conf)) # 결과 저장 경로
conf['output_dir'] = utils.renew_if_path_exist(conf['output_dir'])

# conf 저장
os.makedirs(conf['output_dir'], exist_ok=True)
conf_path = os.path.join(conf['output_dir'], conf['output_dir'].split('/')[-1] + '.json')
utils.save_json(conf, conf_path)

train_info_path = os.path.join(conf['coco_dir_path'], conf['train_info_name'])
valid_info_path = os.path.join(conf['coco_dir_path'], conf['valid_info_name'])

coco_train = COCO(train_info_path)
coco_valid = COCO(valid_info_path)

id2label = utils.get_id2label(conf['classes'])
label2id = utils.get_label2id(id2label)

train = dataset.COCO2dataset(conf['data_dir_path'], coco_train, range(10))
valid = dataset.COCO2dataset(conf['data_dir_path'], coco_valid, range(10))

train_augment_and_transform, validation_transform = dataset.get_transforms()

image_processor = AutoImageProcessor.from_pretrained(
    conf['model_name'],
    do_resize=True,
    size={"max_height": conf['image_size'], "max_width": conf['image_size']},
    do_pad=True,
    pad_size={"height": conf['image_size'], "width": conf['image_size']},
)

# Make transform functions for batch and apply for dataset splits
train_transform_batch = partial(
    dataset.augment_and_transform_batch, transform=train_augment_and_transform, image_processor=image_processor
)
validation_transform_batch = partial(
    dataset.augment_and_transform_batch, transform=validation_transform, image_processor=image_processor
)

train = train.with_transform(train_transform_batch)
valid = valid.with_transform(validation_transform_batch)

eval_compute_metrics_fn = partial(
    train_eval.compute_metrics, image_processor=image_processor, id2label=id2label, threshold=0.0
)

# model = AutoModelForObjectDetection.from_pretrained(checkpoint_path)

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint_path,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

training_args = train_eval.load_train_args(conf)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=image_processor,
    data_collator=dataset.collate_fn,
    compute_metrics=eval_compute_metrics_fn,
)

trainer.train()

log_path = os.path.join(conf['output_dir'], conf['output_dir'].split('/')[-1] + '.txt')
utils.save_log(trainer, log_path)


# train_eval.test_eval(conf, model, image_processor)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Map,Map 50,Map 75,Map Small,Map Medium,Map Large,Mar 1,Mar 10,Mar 100,Mar Small,Mar Medium,Mar Large,Map General trash,Mar 100 General trash,Map Paper,Mar 100 Paper,Map Paper pack,Mar 100 Paper pack,Map Metal,Mar 100 Metal,Map Glass,Mar 100 Glass,Map Plastic,Mar 100 Plastic,Map Styrofoam,Mar 100 Styrofoam,Map Plastic bag,Mar 100 Plastic bag,Map Battery,Mar 100 Battery,Map Clothing,Mar 100 Clothing
1,17.302900,14.632709,0.214400,0.393200,0.195100,-1.000000,0.088800,0.228400,0.195700,0.269600,0.330900,-1.000000,0.170600,0.346700,0.042200,0.127300,0.131400,0.247400,0.338400,0.466700,0.614500,0.700000,0.000000,0.000000,0.092400,0.250000,-1.000000,-1.000000,0.281900,0.525000,-1.000000,-1.000000,-1.000000,-1.000000
2,13.397900,13.588432,0.275000,0.424500,0.326100,-1.000000,0.135000,0.286700,0.241800,0.362400,0.402800,-1.000000,0.172200,0.426400,0.021600,0.154500,0.126700,0.252600,0.356800,0.600000,0.719200,0.800000,0.000000,0.000000,0.172400,0.287500,-1.000000,-1.000000,0.528000,0.725000,-1.000000,-1.000000,-1.000000,-1.000000


There were missing keys in the checkpoint model loaded: ['class_embed.0.weight', 'class_embed.0.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'class_embed.2.weight', 'class_embed.2.bias', 'class_embed.3.weight', 'class_embed.3.bias', 'class_embed.4.weight', 'class_embed.4.bias', 'class_embed.5.weight', 'class_embed.5.bias', 'class_embed.6.weight', 'class_embed.6.bias', 'bbox_embed.0.layers.0.weight', 'bbox_embed.0.layers.0.bias', 'bbox_embed.0.layers.1.weight', 'bbox_embed.0.layers.1.bias', 'bbox_embed.0.layers.2.weight', 'bbox_embed.0.layers.2.bias', 'bbox_embed.1.layers.0.weight', 'bbox_embed.1.layers.0.bias', 'bbox_embed.1.layers.1.weight', 'bbox_embed.1.layers.1.bias', 'bbox_embed.1.layers.2.weight', 'bbox_embed.1.layers.2.bias', 'bbox_embed.2.layers.0.weight', 'bbox_embed.2.layers.0.bias', 'bbox_embed.2.layers.1.weight', 'bbox_embed.2.layers.1.bias', 'bbox_embed.2.layers.2.weight', 'bbox_embed.2.layers.2.bias', 'bbox_embed.3.layers.0.weight', 'bbox_embed.3.layers.0.bias', '

In [8]:
path = '/data/ephemeral/home/Dongjin/level2-objectdetection-cv-07/transformers/Dongjin/1019_resume_train/result/deta-swin-large_3_img_size_720'

In [11]:
model = AutoModelForObjectDetection.from_pretrained(path)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /data/ephemeral/home/Dongjin/level2-objectdetection-cv-07/transformers/Dongjin/1019_resume_train/result/deta-swin-large_3_img_size_720.

In [36]:
def find_checkpoint_path(input_path):
    output_paths = glob(input_path + '/checkpoint*')

    if 1 < len(output_paths):
        raise Exception(f'checkpoint는 1개여야 합니다. {len(output_path)}개 있습니다.')

    output_path = output_paths[0]
    return output_path


In [42]:
model = AutoModelForObjectDetection.from_pretrained(find_checkpoint_path(path))

training_args = train_eval.load_train_args(conf)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=image_processor,
    data_collator=dataset.collate_fn,
    compute_metrics=eval_compute_metrics_fn,
)

trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


RuntimeError: selected index k out of range

In [11]:
conf['saved_model_path']

KeyError: 'saved_model'

: 